<a href="https://colab.research.google.com/github/2303A51758/MITS-Internship/blob/main/medium_level_task_4_ipynyb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 4: On-the-Fly PDF Generation Service
This uses:

FastAPI (for HTTP handling)

ReportLab (for generating PDFs directly in memory)

No files are saved on the server

Returns PDF as a downloadable file in response



1. Install Required Libraries

In [1]:
pip install fastapi uvicorn reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00


2. Python Script: pdf_offer_service.py

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel
from reportlab.pdfgen import canvas
from io import BytesIO
from fastapi.responses import StreamingResponse
from datetime import date

app = FastAPI()

class OfferRequest(BaseModel):
    name: str
    position: str
    duration_weeks: int

@app.post("/generate-offer-letter")
def generate_offer_letter(data: OfferRequest):
    # Create PDF in memory
    buffer = BytesIO()
    p = canvas.Canvas(buffer)

    # Generate PDF content
    p.setFont("Helvetica", 12)
    p.drawString(100, 750, f"Internship Offer Letter")
    p.drawString(100, 720, f"Dear {data.name},")
    p.drawString(100, 700, f"We are pleased to offer you the position of {data.position}.")
    p.drawString(100, 680, f"Your internship duration is {data.duration_weeks} weeks.")
    p.drawString(100, 660, f"Start Date: {date.today().strftime('%B %d, %Y')}")
    p.drawString(100, 640, f"")
    p.drawString(100, 620, f"Sincerely,")
    p.drawString(100, 600, f"Internship Coordinator")

    # Finalize PDF
    p.showPage()
    p.save()
    buffer.seek(0)

    return StreamingResponse(buffer, media_type="application/pdf", headers={
        "Content-Disposition": f"attachment; filename={data.name}_offer_letter.pdf"
    })


3. Run the Server



In [1]:
!uvicorn pdf_offer_service:app --reload

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [2140] using StatReload
ERROR:    Error loading ASGI app. Could not import module "pdf_offer_service".
INFO:     Stopping reloader process [2140]
^C


4. Test It via Swagger UI

In [9]:
POST /generate-offer-letter


NameError: name 'POST' is not defined

In [10]:
import requests

# Define the data payload for the POST request
payload = {
    "name": "John Doe",
    "position": "Software Engineering Intern",
    "duration_weeks": 10
}

# The URL for the FastAPI endpoint
# By default, uvicorn runs on http://127.0.0.1:8000
url = "http://127.0.0.1:8000/generate-offer-letter"

# Send the POST request
try:
    response = requests.post(url, json=payload)

    # Check if the request was successful
    if response.status_code == 200:
        # The response content is the PDF bytes
        # You can save the PDF to a file or handle it as needed
        with open("offer_letter.pdf", "wb") as f:
            f.write(response.content)
        print("PDF generated successfully and saved as offer_letter.pdf")
    else:
        print(f"Error: Request failed with status code {response.status_code}")
        print(response.text)

except requests.exceptions.ConnectionError:
    print("Error: Could not connect to the server.")
    print("Please ensure the uvicorn server is running in another cell or terminal.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error: Could not connect to the server.
Please ensure the uvicorn server is running in another cell or terminal.


In [12]:
{
  "name": "Alice Johnson",
  "position": "Data Science Intern",
  "duration_weeks": 12
}


{'name': 'Alice Johnson',
 'position': 'Data Science Intern',
 'duration_weeks': 12}

Optional Enhancements
Use WeasyPrint or xhtml2pdf for HTML-to-PDF styling

Add company logo/images to PDF using drawImage()

Email the generated PDF (attach it using Gmail API)

